# Red de Géneros Musicales

Para crear una red donde los nodos representen géneros musicales y las aristas representen artistas que pertenecen a múltiples géneros, seguiremos estos pasos:

Obtener los géneros de los artistas.
Construir una red donde los nodos sean géneros.
Crear aristas entre géneros si un artista pertenece a múltiples géneros.

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import random
from sklearn.metrics import precision_score, recall_score

In [2]:
# Configura tus credenciales de la API de Spotify
client_id = 'ea3f14cac2274f8aa7a071614d8f29af'
client_secret = '61ee483d5d5f472daf9cd19938532935'
redirect_uri = 'http://localhost:8888/callback'

In [3]:
# Autenticación en la API de Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri,scope="user-library-read user-read-private"))

In [4]:
# Función para obtener todas las listas de reproducción del usuario
def get_all_playlists():
    playlists = []
    results = sp.current_user_playlists()
    playlists.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        playlists.extend(results['items'])
    return playlists

# Función para obtener géneros de los artistas en lotes
def get_genres_for_artists(artist_ids):
    artist_genres = []
    for i in range(0, len(artist_ids), 50):
        batch = artist_ids[i:i+50]
        artists = sp.artists(batch)
        artist_genres.extend(artists['artists'])
    return artist_genres

# Función para obtener todos los pares de nodos no adyacentes
def non_adjacent_pairs(G):
    pairs = []
    for node in G.nodes():
        for potential_neighbor in G.nodes():
            if node != potential_neighbor and not G.has_edge(node, potential_neighbor):
                pairs.append((node, potential_neighbor))
    return pairs

def top_n_predictions(predictions, n=10):
    sorted_predictions = sorted(predictions, key=lambda x: x[2], reverse=True)
    return sorted_predictions[:n]

In [5]:
# Obtener todas las listas de reproducción del usuario
playlists = get_all_playlists()

# Crear un DataFrame para almacenar las pistas
tracks_data = []

# Obtener todas las pistas de todas las listas de reproducción
for playlist in playlists:
    playlist_id = playlist['id']
    results = sp.playlist_tracks(playlist_id)
    tracks_data.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks_data.extend(results['items'])

# Campos específicos que necesitas
fields = [
    'track.name',
    'track.artists',
    'track.album.name',
    'track.popularity',
    'track.duration_ms'
]

# Normalizar los datos JSON especificando solo los campos necesarios
tracks_df = pd.json_normalize(tracks_data)[fields]

# Expandir la columna de artistas para obtener los nombres y IDs
tracks_df['track.artists'] = tracks_df['track.artists'].apply(lambda x: [{'name': artist['name'], 'id': artist['id']} for artist in x])

# Verificar el DataFrame
tracks_df.head()

,track.name,track.artists,track.album.name,track.popularity,track.duration_ms
0,Hoy El Aire Huele a Ti,"[{'name': 'Luis Miguel', 'id': '2nszmSgqreHSdJ...",20 Años,63,224493
1,Cómo duele,"[{'name': 'Luis Miguel', 'id': '2nszmSgqreHSdJ...",Mis Romances,61,233973
2,Oro De Ley,"[{'name': 'Luis Miguel', 'id': '2nszmSgqreHSdJ...",20 Años,65,238933
3,Todo Y Nada,"[{'name': 'Luis Miguel', 'id': '2nszmSgqreHSdJ...",Segundo Romance,66,219400
4,Tú sólo tú,"[{'name': 'Luis Miguel', 'id': '2nszmSgqreHSdJ...",Amarte Es Un Placer,53,259240


In [7]:
# Obtener los IDs únicos de los artistas
artist_ids = tracks_df['track.artists'].explode().apply(lambda x: x['id'] if isinstance(x, dict) else None).dropna().unique()

# Obtener géneros para todos los artistas
artist_genres = get_genres_for_artists(artist_ids)

# Crear un DataFrame con los géneros de los artistas
genres_df = pd.json_normalize(artist_genres)

# Verificar el DataFrame para asegurarse de que contiene la columna 'genres'
print(genres_df.columns)

# Expandir los géneros en filas separadas
if 'genres' in genres_df.columns:
    genres_exploded = genres_df.explode('genres')
else:
    raise KeyError("La columna 'genres' no se encuentra en el DataFrame")

# Crear un mapa de géneros a artistas
genre_artist_map = genres_exploded.groupby('genres')['name'].apply(set).to_dict()

# Verificar el mapa de géneros a artistas
genre_artist_map

In [ ]:
# Crear una lista de conexiones entre géneros basadas en artistas comunes
genre_edges = []
for genre1, artists1 in genre_artist_map.items():
    for genre2, artists2 in genre_artist_map.items():
        if genre1 != genre2:
            common_artists = artists1.intersection(artists2)
            if common_artists:
                genre_edges.append((genre1, genre2))

# Crear un DataFrame con las conexiones entre géneros
genre_df = pd.DataFrame(genre_edges, columns=['Genre1', 'Genre2'])

## ¿Qué Representa la Red?
Nodos: Géneros musicales.
Aristas: Conexiones entre géneros basadas en artistas comunes. Si dos géneros están conectados por una arista, significa que hay al menos un artista que pertenece a ambos géneros.

In [ ]:
# Crear un grafo vacío
G = nx.Graph()

# Añadir nodos y aristas al grafo basados en las conexiones entre géneros
for _, row in genre_df.iterrows():
    G.add_node(row['Genre1'])
    G.add_node(row['Genre2'])
    G.add_edge(row['Genre1'], row['Genre2'])

# Dibujar el grafo
plt.figure(figsize=(12, 12))
nx.draw(G, with_labels=True, node_size=50, node_color='blue', font_size=8, edge_color='gray')
plt.title('Red de Géneros Musicales Basada en Artistas Comunes')
plt.show()

NameError: name 'genre_df' is not defined

Esta red te permite visualizar cómo están relacionados los géneros musicales en función de los artistas que escuchas en tus listas de reproducción. Es una forma interesante de ver las intersecciones entre diferentes géneros y cómo tu preferencia musical se extiende a través de estos géneros.

In [ ]:
# Identificar y crear el subgrafo con los nodos principales (mayor componente conexa)
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc).copy()

# Dibujar el grafo
plt.figure(figsize=(12, 12))
nx.draw(G, with_labels=True, node_size=50, node_color='blue', font_size=8, edge_color='gray')
plt.title('Red de Géneros Musicales Basada en Artistas Comunes')
plt.show()

Este código garantiza que el grafo G contiene solo la mayor componente conexa, eliminando los nodos periféricos, y procede con el análisis y la visualización de esta red centralizada.

## Análisis Básicos
### 1. Número de Nodos y Aristas

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print(f"Número de géneros (nodos): {num_nodes}")
print(f"Número de conexiones (aristas): {num_edges}")

In [ ]:
# Análisis Adicional - Características Básicas de la Red
density = nx.density(G)
print(f"Densidad de la Red: {density}")

### 2. Grado de los Nodos

In [ ]:
degrees = dict(G.degree())
sorted_degrees = sorted(degrees.items(), key=lambda item: item[1], reverse=True)

# Top 10 géneros con mayor grado
top_10_genres = sorted_degrees[:10]
print("\nTop 10 géneros con mayor grado:")
for genre, degree in top_10_genres:
    print(f"{genre}: {degree}")

### 3. Distribución de Grados

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(list(degrees.values()), bins=range(1, max(degrees.values()) + 1), edgecolor='black')
plt.title("Distribución de Grados")
plt.xlabel("Grado")
plt.ylabel("Número de Géneros")
plt.show()

### 4. Componentes Conexas

In [ ]:
connected_components = list(nx.connected_components(G))
num_components = len(connected_components)
print(f"Número de componentes conexas: {num_components}")
print("Componentes conexas:")
for i, component in enumerate(connected_components):
    print(f"Componente {i + 1}: {component}")

In [ ]:
largest_cc = max(nx.connected_components(G), key=len)
print(f"Tamaño de la mayor componente conexa: {len(largest_cc)}")

## Análisis Adicional
### Centralidad de Grado

In [ ]:
degree_centrality = nx.degree_centrality(G)
sorted_degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
print("\nTop 10 géneros por centralidad de grado:")
for genre, centrality in sorted_degree_centrality[:10]:
    print(f"{genre}: {centrality}")

### Centralidad de Cercanía

In [ ]:
closeness_centrality = nx.closeness_centrality(G)
sorted_closeness_centrality = sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True)
print("\nTop 10 géneros por centralidad de cercanía:")
for genre, centrality in sorted_closeness_centrality[:10]:
    print(f"{genre}: {centrality}")

### Centralidad de Intermediación

In [ ]:
betweenness_centrality = nx.betweenness_centrality(G)
sorted_betweenness_centrality = sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)
print("\nTop 10 géneros por centralidad de intermediación:")
for genre, centrality in sorted_betweenness_centrality[:10]:
    print(f"{genre}: {centrality}")

### Coeficiente de Agrupamiento

In [ ]:
# Coeficiente de Agrupamiento
clustering_coefficient = nx.clustering(G)
sorted_clustering_coefficient = sorted(clustering_coefficient.items(), key=lambda item: item[1], reverse=True)
print("\nTop 10 géneros por coeficiente de agrupamiento:")
for genre, coefficient in sorted_clustering_coefficient[:10]:
    print(f"{genre}: {coefficient}")

In [ ]:
# Dibujar el grafo
plt.figure(figsize=(12, 12))

# Colores de los nodos según su grado (número de conexiones)
node_colors = ['red' if genre in dict(top_10_genres) else 'blue' for genre in G.nodes()]

# Tamaños de los nodos según su grado (número de conexiones)
node_sizes = [degrees[genre] * 20 for genre in G.nodes()]

# Dibujar los nodos y las aristas
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes)
nx.draw_networkx_edges(G, pos, edge_color='gray')
nx.draw_networkx_labels(G, pos, font_size=8, font_color='black')

plt.title('Red de Géneros Musicales Basada en Artistas Comunes\n(Nodos más activos en rojo)')
plt.show()

## Análisis de Predicción de Enlaces
Utilizaremos algunas de las métricas comunes para la predicción de enlaces, como:

In [ ]:
# Obtener los pares de nodos no adyacentes
non_adjacent = non_adjacent_pairs(G)

# Calcular Common Neighbors
common_neighbors = [(u, v, len(list(nx.common_neighbors(G, u, v)))) for u, v in non_adjacent]

# Calcular Jaccard Coefficient
jaccard_coefficient = list(nx.jaccard_coefficient(G, non_adjacent))

# Calcular Resource Allocation Index
resource_allocation_index = list(nx.resource_allocation_index(G, non_adjacent))

# Calcular Preferential Attachment
preferential_attachment = list(nx.preferential_attachment(G, non_adjacent))

### Common Neighbors:
Dos nodos tienen una mayor probabilidad de formar un enlace si comparten muchos vecinos comunes.

In [ ]:
print("Top 10 predicciones de enlaces más probables basadas en Common Neighbors:")
for u, v, p in top_n_predictions(common_neighbors):
    print(f"{u} - {v}: {p}")

### Jaccard Coefficient: 
Mide la similitud entre dos nodos basada en sus vecinos.

In [ ]:
print("\nTop 10 predicciones de enlaces más probables basadas en Jaccard Coefficient:")
for u, v, p in top_n_predictions(jaccard_coefficient):
    print(f"{u} - {v}: {p}")

### Resource Allocation Index: 
Asigna puntuaciones a pares de nodos basadas en la cantidad de recursos compartidos por sus vecinos comunes.

In [ ]:

print("\nTop 10 predicciones de enlaces más probables basadas en Resource Allocation Index:")
for u, v, p in top_n_predictions(resource_allocation_index):
    print(f"{u} - {v}: {p}")


### Preferential Attachment: 
La probabilidad de que dos nodos se conecten es proporcional al producto de sus grados.

In [ ]:
print("\nTop 10 predicciones de enlaces más probables basadas en Preferential Attachment:")
for u, v, p in top_n_predictions(preferential_attachment):
    print(f"{u} - {v}: {p}")